In [3]:
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import  make_scorer
from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
#cargo los datos en el dataframe
dataframe=pd.read_csv('/content/95-cefe-343.csv') #cargar archivo generado por tablaAlelosAB.py
dataframe = dataframe.drop(['id', 'aislamiento', 'CEFI'], axis='columns') #CEFI hace referencia a la columna en donde esta clasificado el aislamiento como sensible, intermedio, resistente. En caso que contengan esa columna eliminar CEFI
print(dataframe)

     OXA-64  rosA  rosA_cp1  macB  ...  OXA-104  OXA-132  OXA-180  CEFEPIME
0       1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0       256
1       1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0        64
2      -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0        32
3      -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0        32
4      -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0        64
..      ...   ...       ...   ...  ...      ...      ...      ...       ...
338    -1.0   1.0      -1.0   1.0  ...     -1.0      1.0     -1.0        32
339    -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0         8
340    -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0      1.0         8
341    -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0         8
342    -1.0   1.0      -1.0   1.0  ...     -1.0     -1.0     -1.0        32

[343 rows x 212 columns]


In [ ]:
#defino las variables "X" y "y"
X = dataframe.drop(['CEFEPIME'], axis='columns')
y = dataframe.iloc[:, -1] 
print(X)
print(y)

In [ ]:
#cambio de variables a categoricas por dummys 
datadummy = pd.get_dummies(X.astype(str))
print(datadummy)

In [ ]:
#Eliminar columnas que indican la ausencia del gen 
for col in datadummy.columns: 
  if col[-4:] == '-1.0':
    print(col)
    del(datadummy[col])

print(datadummy.columns)
print(datadummy.shape)

In [8]:
#convierto la variable de "y" en log2
y_test1 = np.log2(y)
print(y_test1)

0      8.0
1      6.0
2      5.0
3      5.0
4      6.0
      ... 
338    5.0
339    3.0
340    3.0
341    3.0
342    5.0
Name: CEFEPIME, Length: 343, dtype: float64


In [9]:
#hacer prediccion 
Lasso_from_joblib = joblib.load('/content/CEFEPIME-95-rl.pkl') #cargar archivo pkl correspondiente al modelo que desea probar
print("Predicciones")
predictions = 2**np.round(Lasso_from_joblib.predict(datadummy))
print(predictions)
verdaderos = y.to_numpy()
print(verdaderos)
# Por último, calculamos el error cuadrático medio y el estadístico R^2
print ('Error cuadrático medio: %.2f' % mean_squared_error(y_test1, np.log2(predictions)))
print ('Estadístico R_2: %.2f' % r2_score(y_test1, np.log2(predictions)))
print(mean_absolute_error(y_test1, np.log2(predictions)))

Predicciones
[ 64.  64.  64.  64.  64.  32.  64.  64.  64.  32. 128. 256.  64.   4.
  64.  64.  64.  64.  64.  64. 128.  64. 128.  64.  64. 128.  64.  64.
  64.  64.  64.  32.  64.   8.  32.  32.  32.  64.  64.  64.  64.  16.
  64.   4.  16.  32.  64.  32.  32.  64. 256.  32.  64.  64. 256.  32.
  32.  32.  32.  64.  32.  32. 128. 128. 128.  64. 128. 128.  64.  64.
  32. 128.  64.  64.  64.  64.  16.  16.  16.  16.  16.  32.   4.  32.
  16.  32.  32.  32.  16.  32.  32.  32.  32.  32.  32.   4.   8.  16.
  16.  16.  16.  16.  16.  16.  16.  16.   4.  16.  16.   4.   4.  16.
   4.   4.  16.  16.   4.  16.   8.  16.   4.  16.  16.  32.   8.   4.
  16.  16.  16.  16.   4.   4.   4.  16.  32.  16.  16.  16.  16.  16.
  16.  16.  16.  16.  16.   4.  16.   4.  16.  16.   4.   8.  16.  16.
   4.  32.  16.  16.   4.   4.  16.  16.  16.  16.  16.  32.   8.   4.
  16.   4.  16.  16.  16.  16.  16.  16.  16.   8.  16.  32.  16.  16.
   4.   8.  16.  16.  16.  16.  16.  16.  16.  16.   4.  16.  16

In [10]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y, predictions, dill=2):
    frac = np.array(y) / np.array(predictions)
    return ((frac >= 1/dill) & (frac <= dill)).mean()
acc2 = within_dillution(y, predictions, dill=2)
acc4 = within_dillution(y, predictions, dill=4)
print(acc2)
print(acc4)
threshold = 8 # estos valores correspondientes a los puntos de corte del para clasificar como suceptible un genoma (varia segun antibiotico)
threshold1 = 16 # valor correspondiente a los puntos de corte para clasificar como intermedio en el caso que amerite, caso contrario colocamos suceptible
predDF = pd.DataFrame({'predictions':predictions, 'y':y})
vme = len(predDF[(predDF.predictions<=threshold) & (predDF.y>threshold1)])
me = len(predDF[(predDF.predictions>threshold1) & (predDF.y<=threshold)])
print(vme)
print(me)

0.9271137026239067
0.9795918367346939
1
3
